# Electric Vehicle Presence Discovery

## Merge databases

In [2]:
#import packages
import datetime as dat
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylab
import scipy.stats as stats


In [3]:
#load metadata
metadata = pd.read_csv('Data/metadataedited.csv')

In [4]:
#load electricity data
electricity = pd.read_csv('Data/grouped_electricity_data.csv')

In [5]:
metadata.head()

,dataid,building_type,city,state,pv,pv_panel_direction,total_amount_of_pv,amount_of_south_facing_pv,amount_of_west_facing_pv,amount_of_east_facing_pv,house_construction_year,total_square_footage,first_floor_square_footage,second_floor_square_footage,third_floor_square_footage,half_floor_square_footage,lower_level_square_footage,car1
0,2836,Single-Family Home,Austin,Texas,NaN,NaN,NaN,NaN,NaN,NaN,1994.0,2796.0,1476.0,1320.0,0.0,NaN,NaN,NaN
1,2743,Single-Family Home,Austin,Texas,NaN,NaN,NaN,NaN,NaN,NaN,1949.0,1590.0,1590.0,0.0,0.0,NaN,NaN,NaN
2,5323,Single-Family Home,Austin,Texas,NaN,NaN,NaN,NaN,NaN,NaN,2008.0,1185.0,602.0,583.0,0.0,NaN,NaN,NaN
3,8560,Single-Family Home,Austin,Texas,NaN,NaN,NaN,NaN,NaN,NaN,2009.0,2001.0,1435.0,566.0,0.0,NaN,NaN,NaN
4,3313,Single-Family Home,Austin,Texas,NaN,NaN,NaN,NaN,NaN,NaN,2007.0,2001.0,1435.0,566.0,0.0,NaN,NaN,NaN


In [6]:
data = electricity.merge(metadata, on='dataid')

In [7]:
data.head()

,dataid,use,car1_x,gen,grid,building_type,city,state,pv,pv_panel_direction,...,amount_of_west_facing_pv,amount_of_east_facing_pv,house_construction_year,total_square_footage,first_floor_square_footage,second_floor_square_footage,third_floor_square_footage,half_floor_square_footage,lower_level_square_footage,car1_y
0,59,1.752956,NaN,0.000000,1.752956,Single-Family Home,Austin,Texas,NaN,NaN,...,NaN,NaN,2011.0,3825.0,2705.0,1120.0,NaN,NaN,NaN,NaN
1,77,1.638684,NaN,0.486312,1.149250,Single-Family Home,Austin,Texas,yes,South;West,...,1.085,NaN,2009.0,2669.0,1455.0,1214.0,NaN,NaN,NaN,NaN
2,86,1.495249,NaN,0.000000,1.495249,Single-Family Home,Austin,Texas,NaN,NaN,...,NaN,NaN,1948.0,1788.0,1788.0,0.0,0.0,NaN,NaN,NaN
3,93,1.051423,NaN,0.767092,0.279810,Single-Family Home,Austin,Texas,yes,South,...,0.000,NaN,1993.0,2934.0,2934.0,0.0,0.0,NaN,NaN,NaN
4,94,0.691208,NaN,0.000000,0.691208,Single-Family Home,Austin,Texas,NaN,NaN,...,NaN,NaN,1975.0,1713.0,1713.0,0.0,0.0,NaN,NaN,NaN


In [17]:
averagehour = data.drop(['car1_x', 'gen', 'grid', 'total_amount_of_pv', 'pv_panel_direction', 'state', 'amount_of_south_facing_pv', 'amount_of_west_facing_pv', 'amount_of_east_facing_pv', 'first_floor_square_footage', 'second_floor_square_footage', 'third_floor_square_footage', 'half_floor_square_footage', 'lower_level_square_footage'], axis=1)

## clean up NA

In [18]:
averagehour.isnull().sum()

dataid                       0
use                          1
building_type                0
city                         0
pv                          86
house_construction_year     14
total_square_footage         7
car1_y                     180
dtype: int64

In [19]:
fillna = averagehour.fillna(0)
fillna.head()


,dataid,use,building_type,city,pv,house_construction_year,total_square_footage,car1_y
0,59,1.752956,Single-Family Home,Austin,0,2011.0,3825.0,0
1,77,1.638684,Single-Family Home,Austin,yes,2009.0,2669.0,0
2,86,1.495249,Single-Family Home,Austin,0,1948.0,1788.0,0
3,93,1.051423,Single-Family Home,Austin,yes,1993.0,2934.0,0
4,94,0.691208,Single-Family Home,Austin,0,1975.0,1713.0,0


In [20]:
#correct car1_y and pv to be 0 and 1, instead of 0 and yes
fillna.car1_y.replace(('yes'), (1), inplace=True)
fillna.pv.replace(('yes'), (1), inplace=True)

In [21]:
averagehour = fillna
nozero = averagehour.loc[(averagehour['house_construction_year'] != 0)&(averagehour['total_square_footage'] != 0)&(averagehour['use'] !=0)].copy()

In [22]:
averagehour = nozero

In [23]:
averagehour.to_csv('Data/averagehour.csv')